In [33]:
%%writefile project1_mp_1.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <sys/wait.h>

// In this code block, one worker process executes merge sort on a small and large array.

#define SMALL 32
#define LARGE 131072
int smallArr[SMALL];
int largeArr[LARGE];

// Reference: GeeksforGeeks
// Merge function to combine 2 sections of an array
void merge(int arr[], int left, int mid, int right) {
  int i, j, k;
  int n1 = mid - left + 1;
  int n2 = right - mid;

  int leftArr[n1], rightArr[n2];

  for (i = 0; i < n1; i++) {
    leftArr[i] = arr[left + i];
  }
  for (j = 0; j < n2; j++) {
    rightArr[j] = arr[mid + 1 + j];
  }

  i = 0;
  j = 0;
  k = left;

  while (i < n1 && j < n2) {
    if (leftArr[i] <= rightArr[j]) {
      arr[k] = leftArr[i];
      i++;
    }
    else {
      arr[k] = rightArr[j];
      j++;
    }
    k++;
  }

  while (i < n1) {
    arr[k] = leftArr[i];
    i++;
    k++;
  }

  while (j < n2) {
    arr[k] = rightArr[j];
    j++;
    k++;
  }

}

// Reference: GeeksforGeeks
// Recursive merge sort call onto left and right halves of an array
void mergeSort(int arr[], int left, int right) {

  if (left < right) {
      int mid = left + (right - left) / 2;
      mergeSort(arr, left, mid);
      mergeSort(arr, mid+1, right);
      merge(arr, left, mid, right);
  }
}

int main() {

  clock_t start, end;
  double cpu_time;

  // Create small and large random integer arrays
  for (int i = 0; i < SMALL; i++) {
    smallArr[i] = rand() % 31;
  }

  for (int i = 0; i < LARGE; i++) {
    largeArr[i] = rand() % 100001;
  }

  start = clock();
  mergeSort(smallArr, 0, SMALL);
  end = clock();
  cpu_time = ((double)end-start) / CLOCKS_PER_SEC;

  printf("\nSorted 32 integer array: \n");
  for (int i = 0; i < SMALL; i++) {
    printf("%d ", smallArr[i]);
  }
  printf("\nCPU Time to execute Merge Sort 32 integer array with 1 worker process (Multiprocessing): %f\n", cpu_time);

  start = clock();
  mergeSort(largeArr, 0, LARGE);
  end = clock();
  cpu_time = ((double)end-start) / CLOCKS_PER_SEC;

  printf("\nSorted 131072 integer array: \n");
  for (int i = 0; i < LARGE; i++) {
    printf("%d ", largeArr[i]);
  }
  printf("\nCPU Time to execute Merge Sort 131072 integer array with 1 worker process (Multiprocessing): %f\n", cpu_time);


  return 0;
}

Overwriting project1_mp_1.c


In [34]:
%%shell
gcc project1_mp_1.c -o project1_mp_1
./project1_mp_1


Sorted 32 integer array: 
1 2 2 3 4 5 5 6 7 7 7 10 11 12 12 13 13 14 17 18 19 19 22 22 23 28 28 29 29 29 30 30 
CPU Time to execute Merge Sort 32 integer array with 1 worker process (Multiprocessing): 0.000007

Sorted 131072 integer array: 
0 0 0 1 2 3 5 5 6 6 6 7 7 7 8 8 9 9 10 10 10 10 11 15 16 17 17 19 19 21 21 24 24 24 25 26 30 30 32 33 33 35 35 36 37 39 40 41 41 41 41 42 42 44 44 44 45 45 46 46 47 47 48 48 49 50 51 54 54 56 57 58 60 61 61 61 62 65 65 65 66 69 70 72 73 73 73 73 74 74 75 76 76 79 80 80 81 81 82 82 82 82 82 83 83 84 85 86 87 87 87 87 88 89 89 90 91 92 93 94 94 95 95 96 96 97 97 98 98 99 100 100 102 104 104 105 106 106 106 107 110 110 110 113 113 113 114 114 115 115 116 118 119 119 120 120 120 121 121 122 123 123 123 126 127 127 127 128 129 130 132 133 133 135 136 136 136 137 138 140 140 141 142 143 144 144 144 145 146 146 147 147 147 149 150 151 152 153 153 155 155 157 158 158 158 159 160 160 160 161 161 162 165 165 166 169 169 171 173 173 173 175 176 176 178 179 17

In [35]:
%%writefile project1_mp_2_small.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <sys/wait.h>

// In this code block, a two worker processes execute a merge sort on a small array.

#define SMALL 32
int smallArr[SMALL];

typedef struct {
  int *smallArr;
  int left;
  int right;
} SmallArgs;

// Reference: GeeksforGeeks
// Merge function to combine 2 sections of an array
void merge(int arr[], int left, int mid, int right) {
  int i, j, k;
  int n1 = mid - left + 1;
  int n2 = right - mid;

  int leftArr[n1], rightArr[n2];

  for (i = 0; i < n1; i++) {
    leftArr[i] = arr[left + i];
  }
  for (j = 0; j < n2; j++) {
    rightArr[j] = arr[mid + 1 + j];
  }

  i = 0;
  j = 0;
  k = left;

  while (i < n1 && j < n2) {
    if (leftArr[i] <= rightArr[j]) {
      arr[k] = leftArr[i];
      i++;
    }
    else {
      arr[k] = rightArr[j];
      j++;
    }
    k++;
  }

  while (i < n1) {
    arr[k] = leftArr[i];
    i++;
    k++;
  }

  while (j < n2) {
    arr[k] = rightArr[j];
    j++;
    k++;
  }

}

// Reference: GeeksforGeeks
// Recursive merge sort call onto left and right halves of an array
void mergeSort(int arr[], int left, int right) {

  if (left < right) {
      int mid = left + (right - left) / 2;
      mergeSort(arr, left, mid);
      mergeSort(arr, mid+1, right);
      merge(arr, left, mid, right);
  }
}

// Thread function for small array to organize arguments of small array and pass to merge sort
void* mergeSortSmall(void* arg) {
  SmallArgs* args = (SmallArgs*) arg;
  int *smallArr = args->smallArr;
  int left = args->left;
  int right = args->right;

  mergeSort(smallArr, left, right);
}

int main() {

  clock_t start, end;
  double cpu_time;

  // Create small and large random integer arrays
  for (int i = 0; i < SMALL; i++) {
    smallArr[i] = rand() % 31;
  }

  SmallArgs small_args = {smallArr, 0, SMALL};

  // Add a process
  pid_t pid1 = fork();

    if (pid1 == -1) {
        perror("fork");
        exit(EXIT_FAILURE);
    }
    pthread_t process1;
    pthread_t process2;
    if (pid1 == 0) {
        // Process 2
        pthread_create(&process2, NULL, mergeSortSmall, &small_args);

    } else {
        // Process 1
        start = clock();
        pthread_create(&process1, NULL, mergeSortSmall, &small_args);
        wait(NULL);
        end = clock();
        cpu_time = ((double)end-start) / CLOCKS_PER_SEC;
        merge(smallArr, 0, SMALL/2, SMALL);
        printf("\nSorted 32 integer array: \n");
        for (int i = 0; i < SMALL; i++) {
          printf("%d ", smallArr[i]);
        }
        printf("\nCPU Time to execute Merge Sort 32 integer array with 2 worker processes (Multiprocessing): %f\n", cpu_time);
    }

  return 0;
}

Overwriting project1_mp_2_small.c


In [36]:
%%shell
gcc project1_mp_2_small.c -o project1_mp_2_small
./project1_mp_2_small

project1_mp_2_small.c: In function ‘main’:
project1_mp_2_small.c:109:9: warning: implicit declaration of function ‘pthread_create’; did you mean ‘timer_create’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  109 |         pthread_create(&process2, NULL, mergeSortSmall, &small_args);
      |         ^~~~~~~~~~~~~~
      |         timer_create

Sorted 32 integer array: 
0 1 2 2 3 4 5 5 6 7 7 7 10 11 12 12 13 13 14 17 18 19 19 22 22 23 28 28 29 29 29 30 
CPU Time to execute Merge Sort 32 integer array with 2 worker processes (Multiprocessing): 0.000105


In [37]:
%%writefile project1_mp_2_large.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <sys/wait.h>

// In this code block, a two worker processes execute a merge sort on a small array.

#define LARGE 131072
int largeArr[LARGE];

typedef struct {
  int *largeArr;
  int left;
  int right;
} LargeArgs;

// Reference: GeeksforGeeks
// Merge function to combine 2 sections of an array
void merge(int arr[], int left, int mid, int right) {
  int i, j, k;
  int n1 = mid - left + 1;
  int n2 = right - mid;

  int leftArr[n1], rightArr[n2];

  for (i = 0; i < n1; i++) {
    leftArr[i] = arr[left + i];
  }
  for (j = 0; j < n2; j++) {
    rightArr[j] = arr[mid + 1 + j];
  }

  i = 0;
  j = 0;
  k = left;

  while (i < n1 && j < n2) {
    if (leftArr[i] <= rightArr[j]) {
      arr[k] = leftArr[i];
      i++;
    }
    else {
      arr[k] = rightArr[j];
      j++;
    }
    k++;
  }

  while (i < n1) {
    arr[k] = leftArr[i];
    i++;
    k++;
  }

  while (j < n2) {
    arr[k] = rightArr[j];
    j++;
    k++;
  }

}

// Reference: GeeksforGeeks
// Recursive merge sort call onto left and right halves of an array
void mergeSort(int arr[], int left, int right) {

  if (left < right) {
      int mid = left + (right - left) / 2;
      mergeSort(arr, left, mid);
      mergeSort(arr, mid+1, right);
      merge(arr, left, mid, right);
  }
}

// Thread function for large array to organize arguments of large array and pass to merge sort
void* mergeSortLarge(void* arg) {
  LargeArgs* args = (LargeArgs*) arg;
  int *largeArr = args->largeArr;
  int left = args->left;
  int right = args->right;

  mergeSort(largeArr, left, right);
}

int main() {

  clock_t start, end;
  double cpu_time;

  // Create large random integer array
  for (int i = 0; i < LARGE; i++) {
    largeArr[i] = rand() % 100001;
  }

  LargeArgs large_args = {largeArr, 0, LARGE};

  // Add a process
  pid_t pid1 = fork();

    if (pid1 == -1) {
        perror("fork");
        exit(EXIT_FAILURE);
    }
    pthread_t process1;
    pthread_t process2;
    if (pid1 == 0) {
        // Process 2
        pthread_create(&process2, NULL, mergeSortLarge, &large_args);

    } else {
        // Process 1
        start = clock();
        pthread_create(&process1, NULL, mergeSortLarge, &large_args);
        wait(NULL);
        end = clock();
        cpu_time = ((double)end-start) / CLOCKS_PER_SEC;
        merge(largeArr, 0, LARGE/2, LARGE);
        printf("\nSorted 131072 integer array: \n");
        for (int i = 0; i < LARGE; i++) {
          printf("%d ", largeArr[i]);
        }
        printf("\nCPU Time to execute Merge Sort 131072 integer array with 2 worker processes (Multiprocessing): %f\n", cpu_time);
    }

  return 0;
}

Overwriting project1_mp_2_large.c


In [38]:
%%shell
gcc project1_mp_2_large.c -o project1_mp_2_large
./project1_mp_2_large

project1_mp_2_large.c: In function ‘main’:
project1_mp_2_large.c:109:9: warning: implicit declaration of function ‘pthread_create’; did you mean ‘timer_create’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  109 |         pthread_create(&process2, NULL, mergeSortLarge, &large_args);
      |         ^~~~~~~~~~~~~~
      |         timer_create

Sorted 131072 integer array: 
0 40 81 106 209 230 255 257 273 292 331 334 352 428 477 479 486 487 488 497 509 519 529 565 580 607 610 615 645 647 667 668 683 706 719 749 752 774 775 776 785 806 809 816 824 826 837 853 866 874 876 885 904 918 953 954 975 983 1019 1034 1052 1067 1102 1120 1121 1131 1170 1236 1242 1251 1272 1319 1353 1355 1402 1421 1479 1480 1481 1492 1505 1510 1534 1569 1579 1580 1590 1597 1636 1638 1640 1678 1706 1709 1752 1801 1834 1839 1856 1893 1937 1972 2015 2055 2059 2105 2126 2160 2213 2217 2225 2241 2247 2248 2275 2286 2305 2320 2353

In [39]:
%%writefile project1_mp_4_small.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <sys/wait.h>

// In this code block, a two worker processes execute a merge sort on a small array.

#define SMALL 32
int smallArr[SMALL];

typedef struct {
  int *smallArr;
  int left;
  int right;
} SmallArgs;

// Reference: GeeksforGeeks
// Merge function to combine 2 sections of an array
void merge(int arr[], int left, int mid, int right) {
  int i, j, k;
  int n1 = mid - left + 1;
  int n2 = right - mid;

  int leftArr[n1], rightArr[n2];

  for (i = 0; i < n1; i++) {
    leftArr[i] = arr[left + i];
  }
  for (j = 0; j < n2; j++) {
    rightArr[j] = arr[mid + 1 + j];
  }

  i = 0;
  j = 0;
  k = left;

  while (i < n1 && j < n2) {
    if (leftArr[i] <= rightArr[j]) {
      arr[k] = leftArr[i];
      i++;
    }
    else {
      arr[k] = rightArr[j];
      j++;
    }
    k++;
  }

  while (i < n1) {
    arr[k] = leftArr[i];
    i++;
    k++;
  }

  while (j < n2) {
    arr[k] = rightArr[j];
    j++;
    k++;
  }

}

// Reference: GeeksforGeeks
// Recursive merge sort call onto left and right halves of an array
void mergeSort(int arr[], int left, int right) {

  if (left < right) {
      int mid = left + (right - left) / 2;
      mergeSort(arr, left, mid);
      mergeSort(arr, mid+1, right);
      merge(arr, left, mid, right);
  }
}

// Thread function for small array to organize arguments of small array and pass to merge sort
void* mergeSortSmall(void* arg) {
  SmallArgs* args = (SmallArgs*) arg;
  int *smallArr = args->smallArr;
  int left = args->left;
  int right = args->right;

  mergeSort(smallArr, left, right);
}

int main() {

  clock_t start, end;
  double cpu_time;

  // Create small and large random integer arrays
  for (int i = 0; i < SMALL; i++) {
    smallArr[i] = rand() % 31;
  }

  SmallArgs small_args = {smallArr, 0, SMALL};

  // Add a process
  pid_t pid1 = fork();

    if (pid1 == -1) {
        perror("fork");
        exit(EXIT_FAILURE);
    }
    pthread_t process1;
    pthread_t process2;
    pthread_t process3;
    pthread_t process4;
    if (pid1 == 0) {
        // Process 2
        pthread_create(&process2, NULL, mergeSortSmall, &small_args);

    } else {
        // Process 1
        start = clock();
        pthread_create(&process1, NULL, mergeSortSmall, &small_args);
        // Add a process
        pid_t pid2 = fork();

        if (pid2 == -1) {
        perror("fork");
        exit(EXIT_FAILURE);
        }
        if (pid2 == 0) {
            // Process 4
            pthread_create(&process4, NULL, mergeSortSmall, &small_args);

        } else {
            // Process 3
            start = clock();
            pthread_create(&process3, NULL, mergeSortSmall, &small_args);
            wait(NULL);
            end = clock();
            cpu_time = ((double)end-start) / CLOCKS_PER_SEC;
            merge(smallArr, 0, SMALL/2, SMALL);
            printf("\nSorted 32 integer array: \n");
            for (int i = 0; i < SMALL; i++) {
              printf("%d ", smallArr[i]);
            }
            printf("\nCPU Time to execute Merge Sort 32 integer array with 4 worker processes (Multiprocessing): %f\n", cpu_time);
        }
    }

  return 0;
}

Overwriting project1_mp_4_small.c


In [40]:
%%shell
gcc project1_mp_4_small.c -o project1_mp_4_small
./project1_mp_4_small

project1_mp_4_small.c: In function ‘main’:
project1_mp_4_small.c:111:9: warning: implicit declaration of function ‘pthread_create’; did you mean ‘timer_create’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  111 |         pthread_create(&process2, NULL, mergeSortSmall, &small_args);
      |         ^~~~~~~~~~~~~~
      |         timer_create

Sorted 32 integer array: 
0 1 2 2 3 4 5 5 6 7 7 7 10 11 12 12 13 13 14 17 18 19 19 22 22 23 28 28 29 29 29 30 
CPU Time to execute Merge Sort 32 integer array with 4 worker processes (Multiprocessing): 0.000082


In [41]:
%%writefile project1_mp_4_large.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <sys/wait.h>

// In this code block, a two worker processes execute a merge sort on a small array.

#define LARGE 131072
int largeArr[LARGE];

typedef struct {
  int *largeArr;
  int left;
  int right;
} LargeArgs;

// Reference: GeeksforGeeks
// Merge function to combine 2 sections of an array
void merge(int arr[], int left, int mid, int right) {
  int i, j, k;
  int n1 = mid - left + 1;
  int n2 = right - mid;

  int leftArr[n1], rightArr[n2];

  for (i = 0; i < n1; i++) {
    leftArr[i] = arr[left + i];
  }
  for (j = 0; j < n2; j++) {
    rightArr[j] = arr[mid + 1 + j];
  }

  i = 0;
  j = 0;
  k = left;

  while (i < n1 && j < n2) {
    if (leftArr[i] <= rightArr[j]) {
      arr[k] = leftArr[i];
      i++;
    }
    else {
      arr[k] = rightArr[j];
      j++;
    }
    k++;
  }

  while (i < n1) {
    arr[k] = leftArr[i];
    i++;
    k++;
  }

  while (j < n2) {
    arr[k] = rightArr[j];
    j++;
    k++;
  }

}

// Reference: GeeksforGeeks
// Recursive merge sort call onto left and right halves of an array
void mergeSort(int arr[], int left, int right) {

  if (left < right) {
      int mid = left + (right - left) / 2;
      mergeSort(arr, left, mid);
      mergeSort(arr, mid+1, right);
      merge(arr, left, mid, right);
  }
}

// Thread function for large array to organize arguments of large array and pass to merge sort
void* mergeSortLarge(void* arg) {
  LargeArgs* args = (LargeArgs*) arg;
  int *largeArr = args->largeArr;
  int left = args->left;
  int right = args->right;

  mergeSort(largeArr, left, right);
}

int main() {

  clock_t start, end;
  double cpu_time;

  // Create large random integer array
  for (int i = 0; i < LARGE; i++) {
    largeArr[i] = rand() % 100001;
  }

  LargeArgs large_args = {largeArr, 0, LARGE};

  // Add a process
  pid_t pid1 = fork();

    if (pid1 == -1) {
        perror("fork");
        exit(EXIT_FAILURE);
    }
    pthread_t process1;
    pthread_t process2;
    pthread_t process3;
    pthread_t process4;
    if (pid1 == 0) {
        // Process 2
        pthread_create(&process2, NULL, mergeSortLarge, &large_args);

    } else {
        // Process 1
        start = clock();
        pthread_create(&process1, NULL, mergeSortLarge, &large_args);
        // Add a process
        pid_t pid2 = fork();

        if (pid2 == -1) {
        perror("fork");
        exit(EXIT_FAILURE);
        }
        if (pid2 == 0) {
            // Process 4
            pthread_create(&process4, NULL, mergeSortLarge, &large_args);

        } else {
            // Process 3
            start = clock();
            pthread_create(&process3, NULL, mergeSortLarge, &large_args);
            wait(NULL);
            end = clock();
            cpu_time = ((double)end-start) / CLOCKS_PER_SEC;
            merge(largeArr, 0, LARGE/2, LARGE);
            printf("\nSorted 131072 integer array: \n");
            for (int i = 0; i < LARGE; i++) {
              printf("%d ", largeArr[i]);
            }
            printf("\nCPU Time to execute Merge Sort 131072 integer array with 4 worker processes (Multiprocessing): %f\n", cpu_time);
        }
    }

  return 0;
}

Overwriting project1_mp_4_large.c


In [42]:
%%shell
gcc project1_mp_4_large.c -o project1_mp_4_large
./project1_mp_4_large

project1_mp_4_large.c: In function ‘main’:
project1_mp_4_large.c:111:9: warning: implicit declaration of function ‘pthread_create’; did you mean ‘timer_create’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  111 |         pthread_create(&process2, NULL, mergeSortLarge, &large_args);
      |         ^~~~~~~~~~~~~~
      |         timer_create

Sorted 131072 integer array: 
0 0 10 40 46 73 81 106 116 123 175 181 209 230 248 257 268 273 286 292 318 331 334 343 352 357 405 428 463 465 477 479 486 487 488 497 509 519 532 539 546 552 565 576 580 585 603 607 610 630 631 643 645 647 665 666 667 668 670 672 678 683 706 708 715 719 727 749 759 774 775 776 785 793 799 806 806 809 816 824 828 830 837 852 866 874 885 900 904 908 918 938 943 953 954 972 975 977 983 990 1019 1034 1034 1052 1067 1098 1120 1121 1129 1130 1131 1133 1141 1170 1176 1185 1189 1206 1211 1221 1226 1232 1238 1242 1244 1251 1272 1292 1

In [43]:
%%writefile project1_mp_8_small.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <sys/wait.h>

// In this code block, a two worker processes execute a merge sort on a small array.

#define SMALL 32
int smallArr[SMALL];

typedef struct {
  int *smallArr;
  int left;
  int right;
} SmallArgs;

// Reference: GeeksforGeeks
// Merge function to combine 2 sections of an array
void merge(int arr[], int left, int mid, int right) {
  int i, j, k;
  int n1 = mid - left + 1;
  int n2 = right - mid;

  int leftArr[n1], rightArr[n2];

  for (i = 0; i < n1; i++) {
    leftArr[i] = arr[left + i];
  }
  for (j = 0; j < n2; j++) {
    rightArr[j] = arr[mid + 1 + j];
  }

  i = 0;
  j = 0;
  k = left;

  while (i < n1 && j < n2) {
    if (leftArr[i] <= rightArr[j]) {
      arr[k] = leftArr[i];
      i++;
    }
    else {
      arr[k] = rightArr[j];
      j++;
    }
    k++;
  }

  while (i < n1) {
    arr[k] = leftArr[i];
    i++;
    k++;
  }

  while (j < n2) {
    arr[k] = rightArr[j];
    j++;
    k++;
  }

}

// Reference: GeeksforGeeks
// Recursive merge sort call onto left and right halves of an array
void mergeSort(int arr[], int left, int right) {

  if (left < right) {
      int mid = left + (right - left) / 2;
      mergeSort(arr, left, mid);
      mergeSort(arr, mid+1, right);
      merge(arr, left, mid, right);
  }
}

// Thread function for small array to organize arguments of small array and pass to merge sort
void* mergeSortSmall(void* arg) {
  SmallArgs* args = (SmallArgs*) arg;
  int *smallArr = args->smallArr;
  int left = args->left;
  int right = args->right;

  mergeSort(smallArr, left, right);
}

int main() {

  clock_t start, end;
  double cpu_time;

  // Create small and large random integer arrays
  for (int i = 0; i < SMALL; i++) {
    smallArr[i] = rand() % 31;
  }

  SmallArgs small_args = {smallArr, 0, SMALL};

  // Add a process
  pid_t pid1 = fork();

    if (pid1 == -1) {
        perror("fork");
        exit(EXIT_FAILURE);
    }
    pthread_t process1;
    pthread_t process2;
    pthread_t process3;
    pthread_t process4;
    pthread_t process5;
    pthread_t process6;
    pthread_t process7;
    pthread_t process8;
    if (pid1 == 0) {
        // Process 2
        pthread_create(&process2, NULL, mergeSortSmall, &small_args);

    } else {
        // Process 1
        start = clock();
        pthread_create(&process1, NULL, mergeSortSmall, &small_args);
        // Add a process
        pid_t pid2 = fork();

        if (pid2 == -1) {
          perror("fork");
          exit(EXIT_FAILURE);
          }
        if (pid2 == 0) {
            // Process 4
            pthread_create(&process4, NULL, mergeSortSmall, &small_args);

        } else {
            // Process 3
            start = clock();
            pthread_create(&process3, NULL, mergeSortSmall, &small_args);
            // Add a process
            pid_t pid3 = fork();

            if (pid3 == -1) {
              perror("fork");
              exit(EXIT_FAILURE);
              }
            if (pid3 == 0) {
                // Process 6
                pthread_create(&process6, NULL, mergeSortSmall, &small_args);

            } else {
                // Process 5
                start = clock();
                pthread_create(&process5, NULL, mergeSortSmall, &small_args);
                // Add a process
                pid_t pid4 = fork();

                if (pid4 == -1) {
                  perror("fork");
                  exit(EXIT_FAILURE);
                  }
                if (pid4 == 0) {
                    // Process 8
                    pthread_create(&process8, NULL, mergeSortSmall, &small_args);

                } else {
                    // Process 7
                    start = clock();
                    pthread_create(&process7, NULL, mergeSortSmall, &small_args);
                    wait(NULL);
                    end = clock();
                    cpu_time = ((double)end-start) / CLOCKS_PER_SEC;
                    merge(smallArr, 0, SMALL/2, SMALL);
                    printf("\nSorted 32 integer array: \n");
                    for (int i = 0; i < SMALL; i++) {
                      printf("%d ", smallArr[i]);
                    }
                    printf("\nCPU Time to execute Merge Sort 32 integer array with 8 worker processes (Multiprocessing): %f\n", cpu_time);
                }
            }
        }
    }

  return 0;
}

Overwriting project1_mp_8_small.c


In [44]:
%%shell
gcc project1_mp_8_small.c -o project1_mp_8_small
./project1_mp_8_small

project1_mp_8_small.c: In function ‘main’:
project1_mp_8_small.c:115:9: warning: implicit declaration of function ‘pthread_create’; did you mean ‘timer_create’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |         pthread_create(&process2, NULL, mergeSortSmall, &small_args);
      |         ^~~~~~~~~~~~~~
      |         timer_create

Sorted 32 integer array: 
0 1 2 2 3 4 5 5 6 7 7 7 10 11 12 12 13 13 14 17 18 19 19 22 22 23 28 28 29 29 29 30 
CPU Time to execute Merge Sort 32 integer array with 8 worker processes (Multiprocessing): 0.000070


In [45]:
%%writefile project1_mp_8_large.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <sys/wait.h>

// In this code block, a two worker processes execute a merge sort on a small array.

#define LARGE 131072
int largeArr[LARGE];

typedef struct {
  int *largeArr;
  int left;
  int right;
} LargeArgs;

// Reference: GeeksforGeeks
// Merge function to combine 2 sections of an array
void merge(int arr[], int left, int mid, int right) {
  int i, j, k;
  int n1 = mid - left + 1;
  int n2 = right - mid;

  int leftArr[n1], rightArr[n2];

  for (i = 0; i < n1; i++) {
    leftArr[i] = arr[left + i];
  }
  for (j = 0; j < n2; j++) {
    rightArr[j] = arr[mid + 1 + j];
  }

  i = 0;
  j = 0;
  k = left;

  while (i < n1 && j < n2) {
    if (leftArr[i] <= rightArr[j]) {
      arr[k] = leftArr[i];
      i++;
    }
    else {
      arr[k] = rightArr[j];
      j++;
    }
    k++;
  }

  while (i < n1) {
    arr[k] = leftArr[i];
    i++;
    k++;
  }

  while (j < n2) {
    arr[k] = rightArr[j];
    j++;
    k++;
  }

}

// Reference: GeeksforGeeks
// Recursive merge sort call onto left and right halves of an array
void mergeSort(int arr[], int left, int right) {

  if (left < right) {
      int mid = left + (right - left) / 2;
      mergeSort(arr, left, mid);
      mergeSort(arr, mid+1, right);
      merge(arr, left, mid, right);
  }
}

// Thread function for large array to organize arguments of large array and pass to merge sort
void* mergeSortLarge(void* arg) {
  LargeArgs* args = (LargeArgs*) arg;
  int *largeArr = args->largeArr;
  int left = args->left;
  int right = args->right;

  mergeSort(largeArr, left, right);
}

int main() {

  clock_t start, end;
  double cpu_time;

  // Create large random integer array
  for (int i = 0; i < LARGE; i++) {
    largeArr[i] = rand() % 100001;
  }

  LargeArgs large_args = {largeArr, 0, LARGE};

  // Add a process
  pid_t pid1 = fork();

    if (pid1 == -1) {
        perror("fork");
        exit(EXIT_FAILURE);
    }
    pthread_t process1;
    pthread_t process2;
    pthread_t process3;
    pthread_t process4;
    pthread_t process5;
    pthread_t process6;
    pthread_t process7;
    pthread_t process8;
    if (pid1 == 0) {
        // Process 2
        pthread_create(&process2, NULL, mergeSortLarge, &large_args);

    } else {
        // Process 1
        start = clock();
        pthread_create(&process1, NULL, mergeSortLarge, &large_args);
        // Add a process
        pid_t pid2 = fork();

        if (pid2 == -1) {
          perror("fork");
          exit(EXIT_FAILURE);
          }
        if (pid2 == 0) {
            // Process 4
            pthread_create(&process4, NULL, mergeSortLarge, &large_args);

        } else {
            // Process 3
            start = clock();
            pthread_create(&process3, NULL, mergeSortLarge, &large_args);
            // Add a process
            pid_t pid3 = fork();

            if (pid3 == -1) {
              perror("fork");
              exit(EXIT_FAILURE);
              }
            if (pid3 == 0) {
                // Process 6
                pthread_create(&process6, NULL, mergeSortLarge, &large_args);

            } else {
                // Process 5
                start = clock();
                pthread_create(&process5, NULL, mergeSortLarge, &large_args);
                // Add a process
                pid_t pid4 = fork();

                if (pid4 == -1) {
                  perror("fork");
                  exit(EXIT_FAILURE);
                  }
                if (pid4 == 0) {
                    // Process 8
                    pthread_create(&process8, NULL, mergeSortLarge, &large_args);

                } else {
                    // Process 7
                    start = clock();
                    pthread_create(&process7, NULL, mergeSortLarge, &large_args);
                    wait(NULL);
                    end = clock();
                    cpu_time = ((double)end-start) / CLOCKS_PER_SEC;
                    merge(largeArr, 0, LARGE/2, LARGE);
                    printf("\nSorted 131072 integer array: \n");
                    for (int i = 0; i < LARGE; i++) {
                      printf("%d ", largeArr[i]);
                    }
                    printf("\nCPU Time to execute Merge Sort 131072 integer array with 8 worker processes (Multiprocessing): %f\n", cpu_time);
                }
            }
        }
    }

  return 0;
}

Writing project1_mp_8_large.c


In [46]:
%%shell
gcc project1_mp_8_large.c -o project1_mp_8_large
./project1_mp_8_large

project1_mp_8_large.c: In function ‘main’:
project1_mp_8_large.c:115:9: warning: implicit declaration of function ‘pthread_create’; did you mean ‘timer_create’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |         pthread_create(&process2, NULL, mergeSortLarge, &large_args);
      |         ^~~~~~~~~~~~~~
      |         timer_create

Sorted 131072 integer array: 
0 40 81 106 106 106 106 106 123 123 123 123 175 175 175 175 181 181 181 181 209 230 255 257 268 268 268 268 273 292 318 318 318 318 331 334 352 428 465 465 477 479 486 487 488 497 509 519 529 529 565 573 573 573 573 576 576 580 585 603 607 610 615 630 630 630 630 631 631 631 631 645 647 667 668 672 672 678 678 678 678 683 706 708 719 727 727 749 752 752 759 774 775 776 785 793 793 793 793 806 806 809 826 826 828 828 828 828 837 852 853 864 864 864 864 866 874 876 876 885 904 906 906 906 906 918 943 943 943 943 953 954 975 983 